* 단어 추가해야 한다.
* torch dataset으로 변경해야 한다.

# train_test_split

In [1]:
import pandas as pd

In [ ]:
stopword = pd.read_csv("stopword_hangul_label.tsv", sep='\t')

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
x_train, x_test = train_test_split(stopword, test_size=0.1, random_state=0)

NameError: name 'stopword' is not defined

In [4]:
print(len(x_train[x_train["label"] == 1]))
print(len(x_train[x_train["label"] == 0]))

NameError: name 'x_train' is not defined

In [ ]:
print(len(x_test[x_test["label"] == 1]))
print(len(x_test[x_test["label"] == 0]))

In [ ]:
x_train.to_csv("stopword_train.tsv", sep='\t', index=False)

In [ ]:
x_test.to_csv("stopword_test.tsv", sep='\t', index=False)

In [ ]:
!pip install gluonnlp

In [ ]:
!pip install mxnet

In [ ]:
# 2.10.0
!pip install transformers

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import re

import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
from KoBERT.kobert.utils import get_tokenizer
from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
#from transformers.optimization import WarmupLinearSchedule
from transformers import AdamW, get_linear_schedule_with_warmup

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model(device)

using cached model
using cached model


In [10]:
dataset_train = nlp.data.TSVDataset("stopword_train.tsv", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("stopword_test.tsv", field_indices=[1,2], num_discard_samples=1)

In [11]:
# 사전 학습이 완료된
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [12]:
class BERTDataset(Dataset):

    def __init__(self,
                 dataset,
                 sent_idx,
                 label_idx,
                 bert_tokenizer,
                 max_len,
                 pad,
                 pair):
        
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, 
            max_seq_length = max_len, 
            pad = pad, 
            pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [13]:
class BERTClassifier(nn.Module):
    
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate = None,
                 params = None):

        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [14]:
class BERTDataset_infer(Dataset):

    def __init__(self,
                 dataset,
                 sent_idx,
                 bert_tokenizer,
                 max_len,
                 pad,
                 pair):
        
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, 
            max_seq_length = max_len, 
            pad = pad,
            pair = pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i])


In [15]:
# 에폭을 줄였다.
max_len = 80
batch_size = 128
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 30
learning_rate =  5e-5

In [16]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [17]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [18]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [19]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [20]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [21]:
# 전체 11,720에서 0.1부터 warmup_ratio
t_total = len(train_dataloader) * num_epochs # 에폭 감안한 전체 학습 건수
warmup_step = int(t_total * warmup_ratio) # 학습율을 0.001로 했다면, 처음에 학습률을 0이었다가 점점 0.001로 수렴

In [22]:
#scheduler = WarmupLinearSchedule(optimizer, warmup_steps = warmup_step, t_total = t_total
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps=t_total)

In [23]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1) # 최고값과 최고값의 인덱스 반환
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0] 
    return train_acc

In [24]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


model.eval()

save_path = 'v8_model_cpu_0525.pt'
save_only_parameter = 'v8_model_only_parameter_0525.pt'

# cpu로 변환
device = torch.device("cpu")
model.to(device)

# 모델 + 파라미터 모두 저장
torch.save(model, save_path)

# 파라미터만 저장
torch.save(model.state_dict(), save_only_parameter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7128811478614807 train acc 0.515625
epoch 1 batch id 31 loss 0.6792380809783936 train acc 0.5493951612903226
epoch 1 batch id 61 loss 0.41495010256767273 train acc 0.6453637295081968
epoch 1 batch id 91 loss 0.3149496912956238 train acc 0.7108516483516484
epoch 1 batch id 121 loss 0.36051109433174133 train acc 0.7565857438016529

epoch 1 train acc 0.7580686475409836


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.9034598214285714


epoch 2 batch id 1 loss 0.2391837239265442 train acc 0.90625
epoch 2 batch id 31 loss 0.13887296617031097 train acc 0.9032258064516129
epoch 2 batch id 61 loss 0.20248225331306458 train acc 0.9162397540983607
epoch 2 batch id 91 loss 0.20565520226955414 train acc 0.9196428571428571
epoch 2 batch id 121 loss 0.21559692919254303 train acc 0.9231017561983471

epoch 2 train acc 0.9236039959016393


...

epoch 19 batch id 1 loss 0.0004600193351507187 train acc 1.0
epoch 19 batch id 31 loss 0.00016727298498153687 train acc 0.9997479838709677
epoch 19 batch id 61 loss 0.00014473870396614075 train acc 0.9998719262295082
epoch 19 batch id 91 loss 0.00015149451792240143 train acc 0.9999141483516484
epoch 19 batch id 121 loss 0.00022451020777225494 train acc 0.999870867768595

epoch 19 train acc 0.9998719262295082



epoch 19 test acc 0.9481026785714286


epoch 20 batch id 1 loss 0.0002713073045015335 train acc 1.0
epoch 20 batch id 31 loss 0.00016413629055023193 train acc 0.9997479838709677
epoch 20 batch id 61 loss 0.00018961168825626373 train acc 0.9998719262295082
epoch 20 batch id 91 loss 0.00017611682415008545 train acc 0.999742445054945
epoch 20 batch id 121 loss 0.00016059912741184235 train acc 0.99974173553719

epoch 20 train acc 0.9997438524590164



epoch 20 test acc 0.9508928571428571


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type BERTClassifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# 그래프 저장

In [ ]:
torch.save(model, save_path)

In [ ]:
device

In [ ]:
model.to(device)

In [ ]:
save_path = 'v6_model_cpu_0408.pt'
save_only_parameter = 'v6_model_only_parameter_0304.pt'

In [ ]:
# 모델 + 파라미터 모두 저장
torch.save(model, save_path)

In [ ]:
# 파라미터만 저장
torch.save(model.state_dict(), save_only_parameter)

In [ ]:
model = torch.load(save_path)

In [ ]:
model.to("cpu")

In [ ]:
model.to(device)

In [ ]:
torch.save(model, 'v6_model_0304.pt')

In [ ]:
##  그래프 전체 저장
#torch.save(model, save_path)
model = torch.load(save_path)
model.eval()
model.to(device)

## 학습 파라미터만 저장(추천)
#torch.save(model.state_dict(), save_only_parameter) # 파라미터만 저장

In [ ]:
new_load_model = BERTClassifier(bertmodel, dr_rate=0.5)

In [ ]:
new_load_model.state_dict(torch.load(save_only_parameter))

In [ ]:
new_load_model.to('cpu')

In [ ]:
new_load_model.eval()

* SKT BERT Inference 방법

In [1]:
text = [['업무로서는 실수였지만 처리 과정에서는 완벽함으로 이끈 결과를 만들었습니다. 새로운 표준을 만들겠습니다 스타벅스 백화점 본점의 오픈 멤버로서 참여하였습니다.', '0']]

In [2]:
# -> 제대로 토크나이징이 안됨.
infer_data = BERTDataset_infer(text, 0, tok, max_len, True, False)

NameError: name 'BERTDataset_infer' is not defined

In [ ]:
#infer_data_tensor = torch.utils.data.DataLoader(infer_data, batch_size=1, num_workers=1)

In [ ]:
infer_tensor = torch.tensor(next(iter(infer_data))[0]).reshape(1, -1)

In [ ]:
segments_tensors = torch.zeros(len(infer_tensor[0]))
segments_tensors = segments_tensors.reshape(1, -1)

In [ ]:
valid_length = torch.tensor(len(infer_tensor[0]))
valid_length = valid_length.reshape(1, -1)

In [ ]:
infer_tensor = infer_tensor.long().to('cpu')
segments_tensors = segments_tensors.long().to('cpu')
valid_length = valid_length.long().to('cpu')

In [ ]:
with torch.no_grad():
    outputs = model(infer_tensor, valid_length, segments_tensors)

In [ ]:
outputs

# 단건 inference

In [ ]:
# 딥러닝 결과 0 타 회사명 들어가도 괜찮은 것, 1이면 반드시 지원하는 회사명이어야 함.
from bert_stopword_inference import *
text = [['그래서 3학년을 마치고 시공과목 교수님을 찾아가 인턴을 해보고싶다고 부탁드렸고 동탄현장에 인턴실습생으로 들어갈 수 있게 되었습니다.']]
output = BERT_inference(text)

* SKT 라벨 없이 inference 방법

In [ ]:
text = [['업무로서는 실수였지만 처리 과정에서는 완벽함으로 이끈 결과를 만들었습니다. 새로운 표준을 만들겠습니다 스타벅스 백화점 본점의 오픈 멤버로서 참여하였습니다.']]

In [ ]:
# -> 제대로 토크나이징이 안됨.
infer_data = BERTDataset_infer(text, 0, tok, max_len, True, False)

In [ ]:
infer_data = torch.tensor(next(iter(infer_data))[0]).reshape(1, -1)

In [ ]:
segments_tensors = torch.zeros(len(infer_data[0]))
segments_tensors = segments_tensors.reshape(1, -1)

In [ ]:
valid_length = torch.tensor(len(infer_data[0]))
valid_length = valid_length.reshape(1, -1)

In [ ]:
infer_tensor = infer_data.long().to('cpu')
segments_tensors = segments_tensors.long().to('cpu')
valid_length = valid_length.long().to('cpu')

In [ ]:
with torch.no_grad():
    outputs = model(infer_tensor, valid_length, segments_tensors)

In [ ]:
torch.argmax(outputs)

* transformers 기술문서 inference 방법

In [ ]:
input_false = "업무로서는 실수였지만 처리 과정에서는 완벽함으로 이끈 결과를 만들었습니다. 새로운 표준을 만들겠습니다 스타벅스 백화점 본점의 오픈 멤버로서 참여하였습니다."

In [ ]:
tokenized_text = tok.basic_tokenizer(input_false)
indexed_token = tok.convert_tokens_to_ids(tokenized_text)
token_tensor = torch.tensor([indexed_token])

In [ ]:
segments_tensors = torch.zeros(len(indexed_token))
segments_tensors = segments_tensors.reshape(1, -1)

In [ ]:
valid_length = torch.tensor(len(indexed_token))
valid_length = valid_length.reshape(1, -1)

In [ ]:
tokens_tensor = token_tensor.to('cpu')
segments_tensors = segments_tensors.to('cpu')
valid_length = valid_length.to('cpu')

In [ ]:
with torch.no_grad():
    outputs = model(tokens_tensor, valid_length, segments_tensors)

In [ ]:
with torch.no_grad():
    outputs = new_load_model(tokens_tensor, valid_length, segments_tensors)

In [ ]:
outputs[0]

In [ ]:
np.array(torch.argmax(outputs))

# 이하 테스트 및 참고 code

In [ ]:
#1.corpus -> 2. tok -> 3. list
from konlpy.tag import Okt
okt = Okt()


stopword = pd.read_csv("stopword_hangul_label.tsv", sep='\t')

corpus = ""

for i in stopword["sentence"]:
    corpus += i

corpust_okt = okt.morphs(corpus) # 3:47
# 548342
len(corpust_okt)

counter = nlp.data.count_tokens(corpust_okt)

# <실험1>
# 새로 추가된 vocab만 가지고 단어장을 만들었다.
my_vocab = nlp.vocab.BERTVocab(counter, min_freq = 3)

# <실험2> # 15:55 # OOM
# 새로운 것과 기존 vocab혼합
# 8002(기존) + 8380(내)
# issue : 새로운 vocab에는 _가 없다.
merge_vocab = nlp.vocab.BERTVocab(counter, reserved_tokens=vocab)


# stopword = pd.read_csv("stopword_hangul_label.csv")

# stopword_sum = 0
# j = 0

# for i in stopword['sentence']:
#     stopword_sum += len(i)
#     j += 1

# avg = stopword_sum / j
# print(avg)

# import torchtext
# torchtext.vocab.Vocab(dataset_train)